In [5]:
%matplotlib inline
import itertools

import matplotlib.pyplot as plt
import numpy as np
from kifmm_py import KiFmm

# Single Precision - BLAS Field Translations

In [31]:
surface_diff_vec = [0, 1, 2]
svd_threshold_vec = [None, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 2e-2]
depth_vec = [3, 4, 5]
expansion_order_vec = [3, 4, 5, 6]

parameters = list(itertools.product(surface_diff_vec, svd_threshold_vec, depth_vec, expansion_order_vec))

In [30]:
np.random.seed(0)

dim = 3
dtype = np.float32
ctype = np.complex64

# Set FMM Parameters
n_vec = 1
n_crit = None
depth = 3
n_sources = 1000000
n_targets = 1000000
kernel = "laplace" 
field_translation = "blas"  
kernel_eval_type = (
    "eval"  
)

sources = np.reshape(
    np.random.rand(n_sources * dim), (n_sources, dim)
).astype(dtype)
targets = np.reshape(
    np.random.rand(n_targets * dim), (n_targets, dim)
).astype(dtype)
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec)
).astype(dtype)

rel_error = []

for (i, (surface_diff, svd_threshold, depth, expansion_order)) in enumerate(parameters):
    print(f"Computing: {i+1}/{len(parameters)}")
    tmp = [expansion_order]*(depth+1)
    fmm = KiFmm(
        tmp,
        sources,
        targets,
        charges,
        kernel_eval_type,
        kernel,
        field_translation,
        prune_empty=True,
        timed=True,
        svd_threshold=svd_threshold,
        surface_diff=surface_diff,
        depth=depth
    )

    fmm.evaluate()

    leaf = fmm.target_leaves[1]
    found = fmm.potentials(leaf)[0]
    targets_leaf = fmm.target_coordinates(leaf)
    expected = fmm.evaluate_kernel(sources, targets_leaf, charges)

    relative_error = np.abs(expected-found)/expected
    max_relative_error = np.max(relative_error)
    min_relative_error = np.min(relative_error)
    mean_relative_error = np.mean(relative_error)

    rel_error.append((min_relative_error, mean_relative_error, max_relative_error))

Computing: 1/81
Computing: 2/81
Computing: 3/81
Computing: 4/81
Computing: 5/81
Computing: 6/81
Computing: 7/81


KeyboardInterrupt: 